In [53]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [54]:
target_url = "https://www.finder.bupa.co.uk/Consultant/search/?order+=rel&rpp=1&giottoFormFlag_consultant=1"
req_header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive' }
main_page = requests.get(url=target_url, headers=req_header)
main_page.status_code

200

In [55]:
# Check the maximum number of consultants available
main_page = BeautifulSoup(main_page.text, 'html.parser')
main_page = main_page.findAll(id = 'list-displaying')
max_list = int(main_page[0].get_text()[52:57])
print(max_list)

31094


In [56]:
# Initialize variables for requests loop
page_code = []
consultant_name = []
df = pd.read_csv('consultant_list_consolidated.csv')
i = df['Code'].iloc[-1] + 1 # get last stopped page code
print(i)

84458


In [ ]:
# Loop to get the list of page codes
while (len(page_code) != 2000):
    test_url = "https://www.finder.bupa.co.uk/Consultant/View/" + str(i)
    test_request = requests.get(url=test_url, headers=req_header)
    if test_request.status_code == 200:
        page_code.append(i)
        page = BeautifulSoup(test_request.text, 'html.parser')
        page = page.findAll(class_ = 'cnwrpt')
        consultant_name.append(page[0].get_text())
    i = i + 1

In [61]:
# Store results in dataframe
df_new = pd.DataFrame({'Code': page_code, 'Consultant Name': consultant_name})
df_new .head()

,Code,Consultant Name
0,84460,Dr Shyam Balasubramanian
1,84462,Mrs Helen M Caulfield
2,84463,Mr Andrew Baird
3,84467,Mr Niall McGurk
4,84484,Dr Jayashree Basu


In [ ]:
# Append to an existing CSV
# if os.path.exists('consultant_list.csv'):
#   os.remove('consultant_list.csv')
df_new.to_csv('consultant_list_consolidated.csv', mode='a', index=False, header=False) 